In [6]:
from __future__ import division
import argparse
import pandas as pd

# useful stuff
import numpy as np
from scipy.special import expit
from sklearn.preprocessing import normalize

In [2]:
__authors__ = ['Rafaelle Aygalenq','Sarah Lina Hammoutene','Dora Linda Kocsis','Noémie Quéré']
__emails__  = ['b00724587@essec.edu','b00712035@essec.edu','b00714326@essec.edu','b00719656@essec.edu']

In [8]:
def text2sentences(path):
    # feel free to make a better tokenization/pre-processing
    sentences = []
    with open(path) as f:
        for l in f:
            sentences.append( l.lower().split() )
    return sentences

def loadPairs(path):
    data = pd.read_csv(path,delimiter='\t')
    pairs = zip(data['word1'],data['word2'],data['similarity'])
    return pairs


class mySkipGram:
    def __init__(self,sentences, nEmbed=100, negativeRate=5, winSize = 5, minCount = 5):
        raise NotImplementedError('implement it!')

    def train(self,stepsize, epochs):
        raise NotImplementedError('implement it!')

    def save(self,path):
        raise NotImplementedError('implement it!')

    def similarity(self,word1,word2):
        """
        computes similiarity between the two words. unknown words are mapped to one common vector
        :param word1:
        :param word2:
        :return: a float \in [0,1] indicating the similarity (the higher the more similar)
        """
        return dot(normalize(self[word1], norm='l2'), normalize(self[word1], norm='l2'))

    @staticmethod
    def load(path):
        raise NotImplementedError('implement it!')

if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--text', help='path containing training data', required=True)
    parser.add_argument('--model', help='path to store/read model (when training/testing)', required=True)
    parser.add_argument('--test', help='enters test mode', action='store_true')

    opts = parser.parse_args()

    if not opts.test:
        sentences = text2sentences(opts.text)
        sg = mySkipGram(sentences)
        sg.train()
        sg.save(opts.model)

    else:
        pairs = loadPairs(opts.text)

        sg = mySkipGram.load(opts.model)
        for a,b,_ in pairs:
            print(sg.similarity(a,b))

usage: ipykernel_launcher.py [-h] --text TEXT --model MODEL [--test]
ipykernel_launcher.py: error: the following arguments are required: --text, --model


SystemExit: 2

C:\Users\dorak\AppData\Local\conda\conda\envs\my_root\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
from collections import defaultdict
import numpy as np


stToken = '<START>'
endToken = '<END>'
class LangModel:

    def __init__(self,n):
        """ n is size of context"""
        self.n = n
        self.count = {}

    def train(self,sequences):
        """ max likelihood estimation """
        for sequence in sequences:
            seq = tuple([stToken]*self.n + sequence + [endToken])
            for i in xrange(0,len(seq)-self.n):
                ctxt = seq[i:i+self.n]
                if ctxt not in self.count:
                    self.count[ctxt] = defaultdict(int)
                self.count[ctxt] [ seq[i+self.n]] += 1
        ctxts = self.count.keys()
        for ctxt in ctxts:
            options = self.count[ctxt]
            norm = float(sum(options.values()))
            self.count[ctxt] = dict((k,v/norm) for k,v in options.iteritems())
    def initCtxt(self):
        return (stToken,)*self.n


    def sample(self):
        ctxt = self.initCtxt()
        nextToken = None
        while nextToken != endToken:
            options = self.count[ctxt]

            nextToken = np.random.choice(options.keys(),size=1,p=options.values())[0]
            yield nextToken
            ctxt = ctxt[1:] + (nextToken,)


import os,codecs
import spacy

def load(path='/nfs/team/nlp/data/TrumpSpeeches/data/'):
    nlp = spacy.load('en')
    sentences = []
    for filename in os.listdir(path):
        if filename.endswith('.txt'):
            with codecs.open(os.path.join(path,filename),encoding='utf-8') as f:
                f.readline() #ignore first line
                doc = nlp(f.readline())
                for sent in doc.sents:
                    sentences.append( [token.text for token in sent] )
    return sentences


ModuleNotFoundError: No module named 'spacy'